In [1]:
import implicit
from scipy import sparse
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('train.csv')

In [3]:
def create_interaction_matrix(df, user_col, item_col, rating_col, norm=False, threshold=None):
    '''
    Create interaction implicit feedback matrix
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [4]:
us_ban_intersect = create_interaction_matrix(data_train,'user_id', 'item_id', 'like')

In [5]:
user_item_sp = sparse.coo_matrix(us_ban_intersect)
item_user_sp = user_item_sp.T

In [6]:
model_item_item = implicit.lmf.LogisticMatrixFactorization()
model_item_item.fit(item_user_sp)

In [7]:
np.array(model_item_item.recommend(166, user_item_sp.tocsr()))[:, 0].astype('int32')

array([76, 22, 35, 37, 72, 65, 58, 66, 80, 40])

In [28]:
ans = np.zeros((len(data_test), 21)).astype('int32')

for num_, user in enumerate(data_test['user_id'].values):
    ans[num_, 0] = user
    ans[num_, 1:] = np.array(model_item_item.recommend(user, user_item_sp.tocsr(), 20))[:, 0].astype('int32')

ans = pd.DataFrame(ans, columns=['user_id']+list(range(20)))